In [1]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [2]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 3614
Dimensão vocabulario positivo: 3044


In [4]:
# Verificar interseção dos vocabulários

from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))


intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)

print("Dimensão inters: " + str(len(intersect)))

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))
lexicon = list(vocab_neg.union(vocab_pos))

Dimensão vocabulario negativo: 3614
Dimensão vocabulario positivo: 3044
Dimensão inters: 1499
Dimensão vocabulario negativo: 2115
Dimensão vocabulario positivo: 1545


In [5]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(2,2), stop_words= stopwords_pt, binary= True)),
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = True))
                    ])

folds = 7;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.910891089109
Fold  1  - Acuracia:  0.841584158416
Fold  2  - Acuracia:  0.89
Fold  3  - Acuracia:  0.88
Fold  4  - Acuracia:  0.9
Fold  5  - Acuracia:  0.89
Fold  6  - Acuracia:  0.939393939394

Accuracia media:  0.89312416956
Desvio padrão:  0.0276508060364


SVM
Cross Validation:
Fold  0  - Acuracia:  0.861386138614
Fold  1  - Acuracia:  0.891089108911
Fold  2  - Acuracia:  0.86
Fold  3  - Acuracia:  0.92
Fold  4  - Acuracia:  0.93
Fold  5  - Acuracia:  0.8
Fold  6  - Acuracia:  0.878787878788

Accuracia media:  0.877323303759
Desvio padrão:  0.0402150655682

Maxent
Cross Validation:
Fold  0  - Acuracia:  0.881188118812
Fold  1  - Acuracia:  0.891089108911
Fold  2  - Acuracia:  0.89
Fold  3  - Acuracia:  0.92
Fold  4  - Acuracia:  0.95
Fold  5  - Acuracia:  0.83
Fold  6  - Acuracia:  0.888888888889

Accuracia media:  0.893023730945
Desvio padrão:  0.0340505282829


In [12]:
# Avaliando a incidência dos bigramas
cv = CountVectorizer(ngram_range=(1,3))

neg_counts = cv.fit_transform(neg_data["texts"])
neg_bigrams = cv.vocabulary_

pos_counts = cv.fit_transform(pos_data["texts"])
pos_bigrams = cv.vocabulary_

sp = np.sum(pos_counts.toarray(), axis=0)
sn = np.sum(neg_counts.toarray(), axis=0)

pos_bigrams  = {bigram: sp[index] for bigram, index in pos_bigrams.items()}
neg_bigrams  = {bigram: sn[index] for bigram, index in neg_bigrams.items()}

pos_excl_bigrams = {key:value for key, value in pos_bigrams.items() if key not in neg_bigrams}
neg_excl_bigrams = {key:value for key, value in neg_bigrams.items() if key not in pos_bigrams}


In [13]:
from pandas import Series
s = Series(pos_excl_bigrams)
s.nlargest(15)

bolsa família     12
plebiscito         9
primeiro turno     9
deus               8
direitos           7
ibope              7
saúde educação     7
todas mentiras     7
ensino             6
internacional      6
investir           6
américa            5
américa latina     5
aprovação          5
caso eleito        5
dtype: int64

In [82]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

lexicon_bigrams = list(pos_excl_bigrams.keys()) + list(neg_excl_bigrams.keys())

features = FeatureUnion([
                    ("bigram", CountVectorizer(ngram_range=(1,1), stop_words= stopwords_pt, vocabulary  = lexicon)),
                    ("lexicon_vector", CountVectorizer(ngram_range=(2,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds=10)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=10)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.845070422535
Fold  1  - Acuracia:  0.842857142857
Fold  2  - Acuracia:  0.814285714286
Fold  3  - Acuracia:  0.885714285714
Fold  4  - Acuracia:  0.885714285714
Fold  5  - Acuracia:  0.842857142857
Fold  6  - Acuracia:  0.842857142857
Fold  7  - Acuracia:  0.857142857143
Fold  8  - Acuracia:  0.871428571429
Fold  9  - Acuracia:  0.871428571429

Accuracia media:  0.855935613682
Desvio padrão:  0.0214964304664


SVM
Cross Validation:
Fold  0  - Acuracia:  0.816901408451
Fold  1  - Acuracia:  0.857142857143
Fold  2  - Acuracia:  0.814285714286
Fold  3  - Acuracia:  0.8
Fold  4  - Acuracia:  0.814285714286
Fold  5  - Acuracia:  0.842857142857
Fold  6  - Acuracia:  0.8
Fold  7  - Acuracia:  0.728571428571
Fold  8  - Acuracia:  0.757142857143
Fold  9  - Acuracia:  0.757142857143

Accuracia media:  0.798832997988
Desvio padrão:  0.0381546235159
